# Produksjonssystem for FNR

Det første vi gjør er å laste inn nødvendige biblioteker
* matplotlib for plotting
* ipywidgets og IPython for menyer
* toml for å laste inn filer med opsjoner

I tillegg laster vi inn FNR-klassen fra fnr.py

In [ ]:
import toml
from fnr_plott_og_tabell import *

import sys
sys.path.append('py')
import fnr as fnr

%matplotlib inline

# Oppsett av FNR-klassen
Vi setter først opp sti til data og kataloger (for aggregering).
I tillegg setter vi opp en liste med alle fylker vi ønsker å ha med samt aggregeringer som ikke ligger i omkodingskatalogen.
Vi laster dette inn fra en toml-fil i samme mappe.

In [ ]:
config = toml.load('config.toml')
mappings = config.get('mappings')
fylker = config.get('fylker').get('fylker')
data = config.get('filstier').get('data')
katalog = config.get('filstier').get('katalog')

Deretter initialiserer vi FNR-klassen og laster inn data for ønskede årganger. Under vises også hvordan man laster inn en ekstra årgang.

In [ ]:
fnr = fnr.fnr_class(2008, 2019, {'lists': ['pubagg', 'a38np'], 'mappings': mappings}, fylker, data, katalog)
fnr.add_year(2020)

Nå har vi en "instance"/et tilfelle av FNR-klassen. En egenskap ved denne er DataFrame'en med alle dataene.

In [ ]:
fnr.return_selection('pubagg', [2019,2020], ['bnp'], ['2x35', '2x41_43'], [], wide_by='fylke', round_to=0)

# Plotting
FNR-klassen har to metoder (funksjoner) man kan benytte seg av. Den ene heter **return_selecion()**. Man kan skrive *help(return_selection)* for se hvordan denne brukes.
Metoden **return_selection()** kan brukes til å hente ut data og foreksemepel plotte disse som vist under.

In [ ]:
plott_fylke(fnr.df, fnr)

In [ ]:
plott_naering(fnr.df, fnr.agg_dict, fnr)

# Tabulering av data
Man kan også bruke metoden **return_selection()** til å tabulere valgte data.

In [ ]:
lag_tabell(fnr.df, fnr.agg_dict, fnr)

# Prikking av data
Den andre metoden i klassen heter **supress_data()**. Denne prikker data (setter verdiene til "not a number/nan"). Den tar i mot en dictionary med nøklene årgang og en liste med par av næringer og fylker som skal prikkes.

In [ ]:
fnr.suppress_data({2020: [['2x90_97', 'f54'], ['2x85', 'f30']]})
fnr.df

# Ekstra
Eksempel på bruk av **return_selection()**.

In [ ]:
fnr.return_selection('pubagg', [], ['bnp'], [], None, round_to=0, wide_by='fylke', first_diff=True)

In [ ]:
lag_tabell(fnr.df, fnr.agg_dict)

In [ ]:
def plott_naering(df, agg_dict):
    aargang_min = df.index.get_level_values('årgang').min().year
    aargang_max = df.index.get_level_values('årgang').max().year
    variabler = [x for x in df.index.get_level_values('nr_variabler').unique().to_list() if x != 'bnpf']
    aggregeringer = [x for x in df.index.get_level_values('aggregering').unique().to_list() if x != 'naering']
    regionsreformer = ['-2017', '2018-2019', '2020-']

    def lag_aggregater(variabel, aggregering):
        if variabel == 'brin':
            return [x for x in df[df.index.get_level_values('aggregering') == aggregering].index.get_level_values('aggregat').unique().to_list() if x[0] == '8']
        else:
            return [x for x in df[df.index.get_level_values('aggregering') == aggregering].index.get_level_values('aggregat').unique().to_list() if x[0] == '2']

    def lag_fylker(regionsreform):
        if regionsreform == '-2017':
            return ['f01', 'f02', 'f03', 'f04', 'f05', 'f06', 'f07', 'f08', 'f09', 'f10', 'f11', 'f12', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f23']
        if regionsreform == '2018-2019':
            return ['f01', 'f02', 'f03', 'f04', 'f05', 'f06', 'f07', 'f08', 'f09', 'f10', 'f11', 'f12', 'f14', 'f15', 'f50', 'f18', 'f19', 'f20', 'f21', 'f23']
        if regionsreform == '2020-':
            return ['f30', 'f03', 'f34', 'f38', 'f42', 'f11', 'f46', 'f15', 'f50', 'f18', 'f54', 'f21', 'f23']

    aargang_wgt = IntRangeSlider(value=[aargang_min, aargang_max], min=aargang_min, max=aargang_max)
    variabel_wgt = Dropdown(options=variabler, description='Variabel', style={'description_width': '3cm'})
    aggregering_wgt = Dropdown(options=aggregeringer, value=aggregeringer[1], description = 'Aggregering', style={'description_width': '3cm'})
    aggregat_wgt = SelectMultiple(options=lag_aggregater(variabler[0], aggregeringer[1]), value=lag_aggregater(variabler[0], aggregeringer[1]), description = 'Aggregat', style={'description_width': '3cm'}, rows=10)
    regionsreformer_wgt = Dropdown(options=regionsreformer, value=regionsreformer[2], description = 'Regionsreform', style={'description_width': '3cm'})
    fylker_wgt = Select(options=lag_fylker(regionsreformer[2]), value=lag_fylker(regionsreformer[2])[0], description = 'Fylke', style={'description_width': '3cm'}, rows=10)
    stack_wgt = Checkbox(value=True, description='Stable stolper')
    first_diff_wgt = Checkbox(value=False, description='Førstedifferanse')
    wide_by_wgt = Dropdown(options=['årgang', 'nr_variabler', 'aggregat', 'fylke'], value='fylke', description = 'Kolonnevariabel', style={'description_width': '3cm'})
    det_wgt = Checkbox(value=False, description='Vis næringer')

    def oppdater_aggregater(arg):
        aggregat_wgt.options = lag_aggregater(variabel_wgt.value, aggregering_wgt.value)

    def oppdater_fylker(arg):
        fylker_wgt.options = lag_fylker(regionsreformer_wgt.value)

    variabel_wgt.observe(oppdater_aggregater)
    aggregering_wgt.observe(oppdater_aggregater)
    regionsreformer_wgt.observe(oppdater_fylker)

    def skriv_tabell(aargang, aggregering, aggregat, variabel, fylker, stack, forste_diff, bred, det):
        if det:
            aggregering_ = 'naering'
            aggregat_ = []
            for agg in aggregat:
                aggregat_.extend(agg_dict.get(aggregering).get(agg))
        else:
            aggregering_ = aggregering
            aggregat_ = aggregat
        try:
            (
                fnr
                .return_selection(aggregering_, list(range(aargang[0], aargang[1]+1)), [variabel], aggregat_, fylker, wide_by='aggregat', first_diff=forste_diff)
                .data
                .reset_index().set_index('årgang')
                .plot
                .bar(figsize=(15, 7.5), stacked=stack)
            )
            plt.xlabel('Årgang')
            plt.legend(ncol=10)
            plt.grid(axis='y')
            plt.axhline(y=0, color='r', linestyle='-')
            plt.show()
        except TypeError:
            pass

    ui = HBox([VBox([stack_wgt, first_diff_wgt, det_wgt, wide_by_wgt, aargang_wgt, variabel_wgt]), VBox([aggregering_wgt, aggregat_wgt]), VBox([regionsreformer_wgt, fylker_wgt])])
    out = interactive_output(skriv_tabell, {'aargang': aargang_wgt,
                                            'aggregering': aggregering_wgt,
                                            'aggregat': aggregat_wgt,
                                            'variabel': variabel_wgt,
                                            'fylker': fylker_wgt,
                                            'stack': stack_wgt,
                                            'forste_diff': first_diff_wgt,
                                            'bred': wide_by_wgt,
                                            'det': det_wgt})
    display(ui, out)


plott_naering(fnr.df, fnr.agg_dict)

In [ ]:
plott_fylke(fnr.df, fnr.agg_dict)